##[Go to Start](#Start)

#Calculation fo Survival Fraction and TCP for varying dose/frac
Alpha (alpha/beta) will have a range of values.


LQ - Model for individual fraction for patient *p*:
$$SF^p = exp(-\alpha^p d - \beta^p d^2)$$

$$TCP^p = exp(-N^p_0 \cdot S)$$
$$TCP^p = exp(-N^p_0 \cdot exp(-\alpha^p d - \beta^p d^2))$$

After n fractions:
$$SF^p_n = \prod^n_0 exp(-\alpha^p d_n - \beta^p d_n^2)$$
$$TCP^p_n = exp[-N^p_0 \cdot \prod^n_0 exp(-\alpha^p d_n - \beta^p d_n^2)]$$

For P patients:
$$TCP^p_n = \frac{\sum^P_p TCP^p_n}{P}$$
.

## Import necessary modules
Numpy for calcs
Matplotlib for plotting

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.html.widgets import FloatProgress
from IPython.display import display
%matplotlib qt
# qt if plot in seperate window
# inline if plot on page

## Set the number of d.p. displayed in numpy arrays
np.set_printoptions(precision=3)

:0: FutureWarning: IPython widgets are experimental and may change in the future.


### Function to calculate an alpha/beta from a normal distribution
User enters nominal alph/beta and SD
Beta is constant at 0.03 (from lit)
Returned is alpha

####Maybe better to accept an alpha value and a SD...? As this is what is likely published in the lit? Could then remove beta completely?

In [17]:
def alphacalc(alphabeta, sd):
    """Return alphabetanew and alpha from normal distribution as specified by sd.
    Default is alphabeta of 10, and sd of 2, beta = 0.03
    If a negative value is returned it is resampled until positive"""
    
    beta = 0.03
    
    ## fixed beta at 0.03 as default in function
    
    ## get alpha beta to use from normal distribution
    if sd == 0:
        alphabetanew = alphabeta
    else:
        alphabetanew=np.random.normal(loc = alphabeta, scale = sd)
    
    ## make sure a positive value is returned
    while alphabetanew <= 0:
        alphabetanew=np.random.normal(loc = alphabeta, scale = sd)
    
    alpha = beta*alphabetanew
   
    return alpha, beta
## alpha/beta can be calced form the returned alpha and beta values

###Function to give a dose per fraction drawn from a normal distribution

In [18]:
def fracdose(dose, d_shift, d_sd):
    """Return dose_actual from normal distribution around dose (Gy) as specified by sd (%) and shift (%).
    Default is dose = 2Gy, shift = 0%, and sd of 0%
    If a negative value is returned it is resampled until positive
    The standard deviation is of the nominal dose"""
    
    ## get actual dose to use from normal distribution based on shift
    
    dose_shift = dose + (dose*shift/100)
    
    ## if sd is zero, then no change to dose
    if sd == 0:
        dose_actual = dose_shift
        return dose_actual
    
    dose_actual=np.random.normal(loc = dose_shift, scale = (dose*sd/100))
    
    ## make sure a positive value is returned
    while dose_actual <= 0:
        dose_actual=np.random.normal(loc = dose_shift, scale = (dose*sd/100))
    
    return dose_actual

##Function to calculate SF

In [19]:
## Survival Fraction Calculation
def SFcalc(alpha, beta, dose):
    """Return the SF with input values.
    Note this is for a single dose delivery.
    The product of multiple fractions shoudld be taken
    to give overall SF"""
    
    SF = np.exp(-(alpha*dose) - (beta*(dose**2)))
    
    return SF

##Function to Calculate TCP from given SF and starting number of cells

In [20]:
## TCP Calculation absed on cumulative SF
def TCPcalc(sf, n0):
    """Return the TCP with input values.
    Based on cumulative SF and N0"""
    
    TCP = np.exp(-n0*sf)
    
    return TCP

#### Functions to allow defining of input parameters

In [41]:
def population_details(n, alphabeta_use, alphabeta_sd_use):
    
    ## number of patients in population
    n = n
    
    ## alpha/beta mean
    alphabeta_use = alphabeta_use
    
    ## alpha/beta standard deviation
    alphabeta_sd_use = alphabeta_sd_use
    
    return n, alphabeta_use, alphabeta_sd_use


def dose_details(d, d_shift, d_sd):
    
    ## Dose per fraction (Gy)
    d = d
    ## Mean Dose shift (%)
    d_shift = d_shift
    ## dose sd (%)
    d_sd = d_sd
    
    return d, d_shift, d_sd


def calc_details(n0, max_d, dose_of_interest):
    
    ## Initial Number of tumour cells (or density)
    n0 = n0
    
    ## Define Max dose for calcs (Gy) - Generally this will stay fixed to maintain consistency between plots
    max_d = max_d
    
    ## dose level at which to extract statistics
    dose_of_interest = dose_of_interest
    
    return n0, max_d, dose_of_interest

####Function to Create an array containing fraction number and nominal dose

In [22]:
## Calc Number of fractions to get to max dose (note: round up as always want an integer)

def no_frac_nom_doses_array(max_d, d):
    n_frac = np.ceil(max_d/d)

    fractions = np.arange(1,n_frac+1)
    #print(fractions)
    nom_doses = np.arange(d,(d*n_frac)+d, step = d)
    #print(nom_doses)
    return fractions, nom_doses, n_frac

####Function to Create a results array containing the number of patients

In [23]:
## This gives a column with the patient number and makes it easier to check values as going

def create_patients(n):
    
    patients = np.arange(0,n)+1
    patients.shape=(n,1)
    #print(patients)
    return patients

#### Functoin to Calculate and add alpha values to results array

In [24]:
## empty array to store alpha values in

def create_alpha_beta_array(n, alphabeta_use, alpbeta_sd_use):
    alpha_and_beta = np.array([])

    for p in range(0,n):
        alpha_and_beta = np.append(alpha_and_beta,[alphacalc(alphabeta = alphabeta_use, sd=alphabeta_sd_use)])

    ## reshape to get a row per patient
    alpha_and_beta = np.reshape(alpha_and_beta,(n,2))
    #print(alpha_and_beta)
    return alpha_and_beta

#### Function to Calculate a dose per fraction for each patient for x fractions

In [25]:
## Calculate Doses for all patients and all fractions

def doses_array(n, n_frac, d, d_shift, d_sd):
    doses = np.array([])
    for i in range(0,int(n*n_frac)):
        doses = np.append(doses,fracdose(dose = d, shift=d_shift, sd=d_sd))
    doses = np.reshape(doses,(n,n_frac))
    #print(doses)
    return doses

#### Function to combine all useful results arrays into single array

In [26]:
## Combine all results into single array which may be easier to work with for analysis

def combine_results(patients, alpha_and_beta, doses):
    results_wanted = (patients,alpha_and_beta, doses)
    all_results = np.concatenate(results_wanted, axis=1)
    #print(all_results)
    return all_results

####Function to Calc SF for all individual doses with corresponding alpha and then calculate cumulative SF

In [27]:
## Loop through the doses of the first patient (first row [0] of array)

def calc_all_SFs(patients, n, n_frac, alpha_and_beta, doses):
    SFs = np.array([])

    for i in range(0,len(patients)): # loop through each patient (row)
        for j in range(0,int(n_frac)): # loop through each fraction for each patient (col)
            SFs = np.append(SFs,SFcalc(alpha_and_beta[i][0],alpha_and_beta[i][1],doses[i,j]))

    SFs = np.reshape(SFs,(n,n_frac))

    ## GEt cumulative SF for each patient
    SF_cum = np.cumprod(SFs, axis=1)
    return SFs, SF_cum

<a id='Start'></a>
## Start of TCP calc
#[Go to End](#Ending)

## Use created functions to calcualte population TCP from patient population

In [28]:
## Progress bar widget
from IPython.html.widgets import FloatProgress
from IPython.display import display

In [29]:
## allow timing of events
import time

## allow export of results as csv
import csv

In [43]:
#### Parameters to be used in TCP calcs

## make into a crude function for repeated use?

def TCPmulticalcer():
    #t_start = time.time()

    ## patient population
    n, alphabeta_use, alphabeta_sd_use = population_details(n=500,
                                                            alphabeta_use = 10,
                                                            alphabeta_sd_use = 2)
    ## dose delivery variation
    d, d_shift, d_sd = dose_details(d = 2,
                                    d_shift = 1,
                                    d_sd = 0)

    ## TCP calc params
    n0, max_d, dose_of_interest = calc_details(n0 = 1E9,
                                               max_d = 100,
                                               dose_of_interest = 74)

    ##### Only the code above should need altering to vary parameters

    ## Calc Number of fractions and nominal dose per fraction to get to max dose
    fractions, nom_doses, n_frac = no_frac_nom_doses_array(max_d,d)

    ## create array containing number of patients in population
    patients = create_patients(n)

    ## Creat array of alpha and veta values for each patient
    alpha_and_beta = create_alpha_beta_array(n, alphabeta_use, alphabeta_sd_use)

    ## array of doses after each fraction for each patient
    doses = doses_array(n, n_frac, d, d_shift, d_sd)

    ## put all results in an array with a patient on each row
    all_results = combine_results(patients, alpha_and_beta, doses)

    ## Calc cumulative SF for all patients (also return individual fraction SFs)
    SFs, SF_cum = calc_all_SFs(patients, n, n_frac, alpha_and_beta, doses)

    ## Calculate TCP for all individual patients and fractions
    TCPs = TCPcalc(sf = SF_cum, n0=n0)

    ## Calculate population TCP by averaging down the columns
    TCP_pop = np.mean(TCPs, axis = 0)

    frac_of_interest = dose_of_interest/d

    TCP_at_dose_of_interest = TCP_pop[frac_of_interest-1]

    #t_end = time.time()

    #t_total = t_end-t_start
    #print(str(round(t_total,2)) + ' secs for ' + str(n) + ' patients')

    TCPs_of_interest = TCPs[:,frac_of_interest-1]

    TCP_cure = (TCPs_of_interest).sum()
    TCP_cure_percent = 100*TCP_cure/n
    #print('Number of cured patients: ' + str(TCP_cure) + ' (' + str(TCP_cure_percent) + '%)')
    
    return n, TCP_cure_percent

print(n)

NameError: name 'num' is not defined

In [37]:
k = 20 ## number of repeats

t_start = time.time()

## for progres bar display
f = FloatProgress(min=0, max=k-1)
display(f)

## Create array for storing results
results_array = np.array([])

## include parameters used to ensure correct in tabulated results
results_array = np.append(results_array,n)
results_array = np.append(results_array,k)
results_array = np.append(results_array,alphabeta_use)
results_array = np.append(results_array,alphabeta_sd_use)
results_array = np.append(results_array,d)
results_array = np.append(results_array,d_shift)
results_array = np.append(results_array,d_sd)
results_array = np.append(results_array,n0)
results_array = np.append(results_array,max_d)
results_array = np.append(results_array,dose_of_interest)
results_array = np.append(results_array,frac_of_interest)

for i in range(0,k):
    TCP_cure_percent = TCPmulticalcer()
    results_array = np.append(results_array,TCP_cure_percent)
    f.value = i # for updating progress bar
    
#results_array = np.reshape(results_array,(k,1))

t_end = time.time()
t_total = t_end-t_start

#print('Time taken for ' + str(k) + ' repeats of ' + str(n) + ' patients is ' + str(round(t_total,2)) + 'secs')
#print(results_array)

## Append results to CSV file
saveTCPasCSV(filename = 'TCPresults.csv')

print('Completed and saved')


NameError: name 'n' is not defined

In [127]:
d_sd


0.5

In [119]:
## append results to text file.

def saveTCPasCSV(filename):
    ar = results_array

    fl = open(filename, 'a', newline='\n')

    writer = csv.writer(fl)
    writer.writerow(ar)

    fl.close()

## Calculate Number of cured patients (TCP>0.999)

This agrees with the TCP from the plot

In [25]:
##made into crude function for multiple use in loops


    

### For 1000 patient 70.5% have TCP>0.5 for 0 dose shift
### This reduces to 64.7% for a -2% dose shift.
### This means the gamma value is approx 3.

## Display Plot of TCPs

In [22]:
## Plot of individual and population TCPs
    
no_ind_plots = 50

## individual plots cannot be more than total patients
if(no_ind_plots>n):
    no_ind_plots=n

## want to select the individual plots randomly from those calcualted...
ind_plots = np.random.choice(len(TCPs),no_ind_plots, replace=False)

## individuals (specified number of plots chosen)
for i in ind_plots:
    plt.plot(nom_doses,TCPs[i], color = 'grey', alpha = 0.5)
## population
plt.plot(nom_doses,TCP_pop, color='black', linewidth='2', label='Population TCP')

## plot formatting
plt.xlim(0,max(nom_doses))
plt.ylim(0,1.0)
plt.xlabel('Dose (Gy)')
plt.ylabel('TCP')
plt.title('TCPs')
plt.legend(loc = 'best', fontsize = 'medium', framealpha = 1)
plt.axvline(dose_of_interest, color = 'black', ls='--',)
plt.axhline(TCP_pop[frac_of_interest-1], color='black', ls='--')

## add labels with TCP at dose of interest
text_string = ('Pop. TCP = ' + str(round(TCP_at_dose_of_interest,4)) + ' at ' + str(dose_of_interest) + 'Gy')
plt.text(5,0.3,text_string, backgroundcolor='white')

plt.show()

### Get population TCP at dose of interest

Want to repeat this for a number of populations to enable calcualtion of the potential spread of the TCP

## Histogram to display spread of TCP for all patients at dose of interest

In [17]:
## show the range of TCPs at a given fraction number (after a given nominal dose)

plt.hist(TCPs[:,frac_of_interest-1], bins=50, color='grey')
plt.show()
#print(np.mean(TCPs[:,frac_of_interest-1]))

## want to calcualte how the proportion that have TCP = 1.
## Is this just the same as reading off the TCP_pop value?

###Example plot of the different doses selected

In [82]:
## Could get the doses for each fraction in one go. Then calc all the SF in one go, then the TCP
dosevals = np.array([])
for i in range(0,10000):
    dosevals = np.append(dosevals,fracdose(dose = 2, shift=-25, sd=10))
    
plt.hist(dosevals, bins=30)
plt.xlim(0,3)

mean = np.mean(dosevals)
stdev = np.std(dosevals)
print("mean (Gy): " + str(mean))
print("stdev (Gy): " + str(stdev))

mean (Gy): 1.50178621074
stdev (Gy): 0.201194338306


The plot above is the distribution of TCPs after 30 fractions
It can be seen that most patients ahve a TCP of 1 or 0.
This is due to the steep individual TCP curves.
There is a very fine line between a patient being 'cured' or 'not cured'.
So small changes in dose potentialy large impact on an individual patient.

## So Far...
Have:
- Patient number
- Alpha
- Beta (default value in function)
- Dose per fraction for all fractions
- SF after each fraction and cumulative
- TCP Calculated from the Cumulative SF for each individual patient.
- Array of fraction number and doses
- Get overall TCP from average of all patients.
- Plots of all TCPs

To do:
- Turn into large function?

<a id='Ending'></a>

##[Go to Start](#Start)

### Experimenting with lognormal distribution
More practical to use normal distribution as gradient unlikely ot be hugely different, and I am interested in the dose variaiton more than the a/b variation which will remain constant for the population.

In [ ]:
a = np.array([])
b = np.array([])

for i in range(0,10000):
    a = np.append(a,np.random.normal(loc = 10, scale = 2))
    b = np.append(b,np.random.lognormal(mean = np.log(10), sigma = np.log(2)))

print(np.mean(a))
print(np.mean(b))

plt.hist(a, label='a', bins=100, alpha=0.5)
plt.hist(b, label='b', bins=100, alpha=0.5)
plt.legend()
plt.xlim(0,50)
plt.ylim(0,8000)